# Global sea level budget discussion

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [70]:
# package
import os
import numpy as np
import plotly
import plotly.graph_objs as go
import xarray as xr
import sys
import datetime
# script
from ncdf_io import ncdf_io
from regression import regress
from time_convert import time_convert,tarray_month

### File location

In [71]:
aisfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Antarctica/slf.t60.s300.m.deg1.scaMB_decom_multi.IVANS_IJ05_IMBIE.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
gisfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Greenland/slf.t60.s300.m.deg1.scaMB_decom_multi.GERUO_ICE5_COMP.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
lwmfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Landwater/slf.f.t60.s300.m.deg1.scaCLM4.5bgc_gict_hydt_ind_decom_multi.GERUO_ICE5_COMP.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
aodfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/GAC/slf.t100.s0.m.deg1.rf.200204_201410.regress.regts.agmask.-66_66.nc'
glofile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Global/slf.landf.t60.s300.m.deg1.scaCLM4.5bgc_gict_hydt_ind_decom_multi_MB_decom_multi.IVANS_IJ05_IMBIE_GERUO_ICE5_COMP.GAC.t100.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
agofile='/Users/joedhsu/Research/Rsync/Data/Argo_Scripps/RG_ArgoClim_StericH_ano_swTEOS10.200401_201612.regress.regts.-66_66.nc'
altfile='/Users/joedhsu/Research/Rsync/Data/Altimetry/AVISO/monthly_mean_dt_upd/ncfiles/data.process.2015.0217vers.CMEMS/dt_global_allsat_msla_h_giay.0.5deg.20to7.199301_201512.regress.regts.agmask.-66_66.nc'

### To include every time series

In [40]:
files=[aisfile,gisfile,lwmfile,aodfile,glofile,agofile,altfile]

varname=['anoSLF_SumPacAtlInd','anoSLF_SumPacAtlInd','anoSLF_SumPacAtlInd',
         'anoSLF_SumPacAtlInd','anoSLF_SumPacAtlInd','anoSH_SumPacAtlInd',
         'anoSL_SumPacAtlInd']

varname2=['ais','gis','lwm',
          'aod','totmass','steric',
          'totsl']

varname2_long=['Antarctic Ice Sheet','Greenland Ice Sheet','Land Water Mass',
               'Atmospheric water vapor','Total Mass','Total Steric',
               'Total Sea Level']

data=[]
dict_gts={}

# new time stamp
tstamp=tarray_month([2004,1],[2014,10])

for i in range(len(files)):
    # read the netcdf file
    ncfile=ncdf_io(files[i],verbose=0)
    ds=ncfile.read_ncdf2xarray()
    
    # old time stamp
    yeardate=np.array(ds['yeardate'])
    tdict=time_convert(yeardate).year2year_mon()
    year=tdict['year']
    month=tdict['month']
    
    # assign NaN to the missing month 
    gts=np.zeros([len(tstamp)])+np.float('nan')

    # creating the new xarray dataset with new timestamp
    for ii in range(len(year)):
        tsig=datetime.datetime(year[ii],month[ii],15)
        ind=np.where(tstamp==tsig)[0]
        gts[ind]=np.round(np.array(ds[varname[i]])[ii],decimals=1)
    gts[:]=gts[:]-gts[0]
    dict_gts[varname2[i]]=(['time'],gts[:])          
    ds_corr = xr.Dataset(dict_gts,coords={'time': tstamp})
    
    # create list of string base on timestamp
    date_str=['%0.4i-%0.2i'%(tstamp[j].year,tstamp[j].month) for j in range(len(tstamp))]

    # element for plotly line
    gplt = go.Scatter(name=varname2_long[i],
                      x=date_str,
                      y=np.array(ds_corr[varname2[i]]),
                      connectgaps=True,
                      line=dict(width = 2,dash = 'solid'))
    data.append(gplt)
    
# layout for the plotly figure
layout = dict(
            title = 'Global Mean Sea Level',
            showlegend = True,
            calendar = "gregorian",
            xaxis= dict(title='Year'),
            yaxis= dict(title='Global Mean Sea Level (mm)')
             )

# output netcdf file 
ds_corr.to_netcdf('./data/gmsl.nc')

# py.iplot(dict(data=data,layout=layout),filename='plot_gsl')
# # plotly.offline.plot(dict(data=data,layout=layout),filename='plot_gsl.html'\
# #                          ,include_plotlyjs=False,output_type ='div' )
plotly.offline.plot(dict(data=data,layout=layout),filename='./figures/plot_all.html')

'file:///Users/joedhsu/Github_public/data_visualization.dir/figures/plot_all.html'

### To only include the mass steric and total

In [53]:
# Dealing one file at a time 
import xarray as xr
import sys
from time_convert import tarray_month
import datetime

files=[glofile,agofile,altfile]
varname=['anoSLF_SumPacAtlInd','anoSH_SumPacAtlInd','anoSL_SumPacAtlInd']
varname2=['totmass','totsteric','totsl']
varname2_long=['Total Mass','Total Steric','Total Sea Level']
linestyle=['dash','dash','solid']


data=[]
dict_gts={}

# new time stamp
tstamp=tarray_month([2004,1],[2014,10])

for i in range(len(files)):
    # read the netcdf file
    ncfile=ncdf_io(files[i],verbose=0)
    ds=ncfile.read_ncdf2xarray()
    
    # old time stamp
    yeardate=np.array(ds['yeardate'])
    tdict=time_convert(yeardate).year2year_mon()
    year=tdict['year']
    month=tdict['month']
    
    # assign NaN to the missing month 
    gts=np.zeros([len(tstamp)])+np.float('nan')

    # creating the new xarray dataset with new timestamp
    for ii in range(len(year)):
        tsig=datetime.datetime(year[ii],month[ii],15)
        ind=np.where(tstamp==tsig)[0]
        gts[ind]=np.round(np.array(ds[varname[i]])[ii],decimals=1)
    gts[:]=gts[:]-gts[0]
    dict_gts[varname2[i]]=(['time'],gts[:])          
    ds_corr = xr.Dataset(dict_gts,coords={'time': tstamp})
    
    # create list of string base on timestamp
    date_str=['%0.4i-%0.2i'%(tstamp[j].year,tstamp[j].month) for j in range(len(tstamp))]

    # element for plotly line
    gplt = go.Scatter(name=varname2_long[i],
                      x=date_str,
                      y=np.round(np.array(ds_corr[varname2[i]]),decimals=1),
                      connectgaps=True,
                      line=dict(width = 2,dash = linestyle[i]))
    data.append(gplt)

    
# include the mass+steric 
gplt = go.Scatter(name='%s + %s'%(varname2_long[0],varname2_long[1]),
                  x=date_str,
                  y=np.round(np.array(ds_corr[varname2[0]]+ds_corr[varname2[1]]),decimals=1),
                  connectgaps=True,
                  line=dict(width = 2,dash = 'solid'),
                  visible='legendonly')
data.append(gplt)
    
    
# layout for the plotly figure
layout = dict(
            title = 'Global Mean Sea Level',
            showlegend = True,
            calendar = "gregorian",
            xaxis= dict(title='Year'),
            yaxis= dict(title='Global Mean Sea Level (mm)')
             )

# py.iplot(dict(data=data,layout=layout),filename='plot_gsl')
# plotly.offline.plot(dict(data=data,layout=layout),filename='plot_mass_steric_tot.html'\
#                          ,include_plotlyjs=False,output_type ='div' )
plotly.offline.plot(dict(data=data,layout=layout),filename='./figures/plot_mass_steric_tot.html')


'file:///Users/joedhsu/Github_public/data_visualization.dir/figures/plot_mass_steric_tot.html'

### Plotting the mass components

In [57]:
aisfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Antarctica/slf.t60.s300.m.deg1.scaMB_decom_multi.IVANS_IJ05_IMBIE.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
gisfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Greenland/slf.t60.s300.m.deg1.scaMB_decom_multi.GERUO_ICE5_COMP.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
lwmfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Landwater/slf.f.t60.s300.m.deg1.scaCLM4.5bgc_gict_hydt_ind_decom_multi.GERUO_ICE5_COMP.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
aodfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/GAC/slf.t100.s0.m.deg1.rf.200204_201410.regress.regts.agmask.-66_66.nc'
glofile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Global/slf.landf.t60.s300.m.deg1.scaCLM4.5bgc_gict_hydt_ind_decom_multi_MB_decom_multi.IVANS_IJ05_IMBIE_GERUO_ICE5_COMP.GAC.t100.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'

ais=ncdf_io(aisfile,verbose=0)
gis=ncdf_io(gisfile,verbose=0)
lwm=ncdf_io(lwmfile,verbose=0)
aod=ncdf_io(aodfile,verbose=0)
glo=ncdf_io(glofile,verbose=0)

In [58]:
ds_ais=ais.read_ncdf2xarray()
ds_gis=gis.read_ncdf2xarray()
ds_lwm=lwm.read_ncdf2xarray()
ds_aod=aod.read_ncdf2xarray()
ds_glo=glo.read_ncdf2xarray()

### Plotting the global mean sea level contribution from mass changes 

In [61]:
import plotly
import plotly.graph_objs as go
from time_convert import *

plotly.offline.init_notebook_mode(connected=True)

yeardate=np.array(ds_ais['yeardate'])
dict1=time_convert(yeardate).year2year_mon()
# dict1=year2year_mon(yeardate)
yeardate_str=dict1['string']
gais=np.round(np.array(ds_ais['anoSLF_global'])-np.array(ds_ais['anoSLF_global'])[0],decimals=1)
ggis=np.round(np.array(ds_gis['anoSLF_global'])-np.array(ds_gis['anoSLF_global'])[0],decimals=1)
glwm=np.round(np.array(ds_lwm['anoSLF_global'])-np.array(ds_lwm['anoSLF_global'])[0],decimals=1)
gaod=np.round(np.array(ds_aod['anoSLF_global'])-np.array(ds_aod['anoSLF_global'])[0],decimals=1)
gglo=np.round(np.array(ds_glo['anoSLF_global'])-np.array(ds_glo['anoSLF_global'])[0],decimals=1)


gplt_ais = go.Scatter(name='Antarctic ice sheet',
                         x=yeardate_str,
                         y=gais)
gplt_gis = go.Scatter(name='Greenland ice sheet',
                         x=yeardate_str,
                         y=ggis)
gplt_lwm = go.Scatter(name="Land water storage",
                         x=yeardate_str,
                         y=glwm)
gplt_aod = go.Scatter(name="Atmosphere water vapor",
                         x=yeardate_str,
                         y=gaod)
gplt_glo = go.Scatter(name="Sum",
                         x=yeardate_str,
                         y=gglo)
layout = dict(
            title = 'Global Mean Sea Level',
            showlegend = True,
            calendar = "gregorian",
            xaxis= dict(title='Year'),
            yaxis= dict(title='Global Mean Sea Level (mm)')
             )
    
data=[gplt_ais,gplt_gis,gplt_lwm,gplt_aod,gplt_glo]

# py.iplot(dict(data=data,layout=layout),filename='plot_gsl')
# plotly.offline.plot(dict(data=data,layout=layout),filename='plot_gsl.html'\
#                          ,include_plotlyjs=False,output_type ='div' )
plotly.offline.plot(dict(data=data,layout=layout),filename='./figures/plot_gsl.html')

'file:///Users/joedhsu/Github_public/data_visualization.dir/figures/plot_gsl.html'

### Use the class regress 


[first cell](#first cell)

In [63]:
reg=regress(yeardate, axis_rel=0)
gglo_reg=reg.multivar_regress(gglo,predef_var='semisea_sea_lin')
gglo_model=np.round(gglo_reg['model'],decimals=1)
print '---------------------------'
print 'Regress result:'
print reg.dm_order[:] 
print gglo_reg['beta'][0]
print gglo_reg['se'][0]
print gglo_reg['rmse']

---------------------------
Regress result:
['con', 'lin', 'anncos', 'annsin', 'semianncos', 'semiannsin']
-3517.428521355614
82.00474202514879
1.682196123268106


### Testing the list_variates

running the <a id='first cell'> **cell above** </a>

In [66]:
reg=regress(yeardate, axis_rel=0.)
gglo_reg=reg.multivar_regress(gglo,list_variates= gglo_reg['list_variates'])
gglo_model=np.round(gglo_reg['model'],decimals=1)
print '---------------------------'
print 'Regress result:'
print reg.dm_order[:] 
print gglo_reg['beta'][0]
print gglo_reg['se'][0]     
print gglo_reg['rmse']

---------------------------
Regress result:
['var1', 'var2', 'var3', 'var4', 'var5', 'var6']
-3517.428521355614
82.00474202514879
1.682196123268106


### Plot the time series removing the regressed model

In [67]:
### seasonal and semiseasonal 
fre_list=[2,3,4,5]
def_model1=np.zeros(len(yeardate))
print '---------------------------'
print 'Model includes:'
for i in fre_list:
    def_model1+=np.array(gglo_reg['list_variates'][i])*gglo_reg['beta'][i]
    print ' ',reg.dm_order[i]  

---------------------------
Model includes:
  var3
  var4
  var5
  var6


In [68]:
### linear
fre_list=[0,1]
def_model2=np.zeros(len(yeardate))
print '---------------------------'
print 'Model includes:'
for i in fre_list:
    def_model2+=np.array(gglo_reg['list_variates'][i])*gglo_reg['beta'][i]
    print ' ',reg.dm_order[i]  

---------------------------
Model includes:
  var1
  var2


In [69]:
gplt_glo = go.Scatter(name="Barystatic",
                         x=yeardate_str,
                         y=gglo)
gplt_glo_reg = go.Scatter(name="Remove seasonal",
                         x=yeardate_str,
                         y=gglo-def_model1)
gplt_glo_lin = go.Scatter(name="Trend %0.2f +- %0.2f mm/yr"%(gglo_reg['beta'][1],gglo_reg['se'][1]),
                         x=yeardate_str,
                         y=def_model2,
                      line=dict(width = 2,dash = 'dash'))
layout = dict(
            title = 'Global Mean Sea Level',
            showlegend = True,
            calendar = "gregorian",
            xaxis= dict(title='Year'),
            yaxis= dict(title='Global Mean Sea Level (mm)')
             )
    
data=[gplt_glo,gplt_glo_reg,gplt_glo_lin]

# py.iplot(dict(data=data,layout=layout),filename='plot_gsl_reg')
# plotly.offline.plot(dict(data=data,layout=layout),filename='plot_gsl_reg.html'\
#                          ,include_plotlyjs=False,output_type ='div' )
plotly.offline.plot(dict(data=data,layout=layout),filename='./figures/plot_gsl_reg_mean.html')

'file:///Users/joedhsu/Github_public/data_visualization.dir/figures/plot_gsl_reg_mean.html'

### Plotting the global mean sea level contribution from mass changes 